In [1]:
import scapy.all as sp
import pandas as pd
import os
import sys
import random
import math
import numpy as np
import random

In [2]:
def determining_normal_rate_packets(pcap_file ):
    time_start = math.inf
    time_end = 0
    total_packets = 0
    total_rate = 0
    with sp.PcapReader(pcap_file) as trace:
        for packet in trace:
            total_packets += 1
            time = packet.time
            time_start = min(time_start, time)
            time_end = max(time, time_end)
            total_rate += len(packet)

    total_time = time_end - time_start
    return float(total_packets/total_time)

In [3]:
#### simulating a real life scenario
def arrange_by_arrival_time(pcap_file):
    packets = []
    times = []
    with sp.PcapReader(pcap_file) as trace:
        for packet in trace:
            packets.append(packet)
            times.append(packet.time)

    sorted_idx = np.argsort(times)
    return_packets = []
    for i in sorted_idx:
        return_packets.append(packets[i])
    return return_packets

In [99]:
def evaluate_buffer(ave_packets, injected_packets, buffer_level, fluctuation_rate):
    overhead_level = (sum(injected_packets)/len(injected_packets) - ave_packets)/ave_packets
    overshoot = 0
    for i in injected_packets:
        if i>buffer_level*ave_packets*(1+fluctuation_rate):
            overshoot+=1
    masking_score = (len(injected_packets) - overshoot)/len(injected_packets)
    return overhead_level, masking_score

In [126]:
def simulate_by_packets(packets_rates, packets_sorteds, fluctuation_rate, queue_size, delay_times):
    return_packets = []
    injected_packets_num = []
    num_devices = len(packets_sorteds)
    #### fluctuation rate determines the upper and lower bound the number of packets per second should be
    ##### if the number of packets per second goes greater than the upper bound, then some packets will be queued
    ##### if the number of packets per second is smaller than the lower bound, then packets will be taken from  dynamic queue
    #### first, then the static queue(fake packets)
    num_packets = [len(packets_sorteds[i]) for i in range(num_devices)]
    dbs = [[] for i in range(num_devices)]
    cbs=[]
    
    
    
    cbs.append(packets_sorteds[0][0].copy())
    start_time = math.floor(packets_sorteds[0][0].time)
    end_time = 0
    for i in range(num_devices):
        end_time = max(end_time, math.ceil(packets_sorteds[i][-1].time))
        
    indexes =  [0 for i in range(num_devices)]
    

    for time in range(start_time, end_time):
        print("time")
        print(time-start_time)
        num_injected =  0
        udps = 0
        for i in range(num_devices):
            udp = []
            non_udp = []
            while indexes[i]<num_packets[i]:
                packet = packets_sorteds[i][indexes[i]]
                ##### only considering the packets that whose time lie between time and time +1. 
                ##### This resembles real life situation that the packets are incoming
                if packet.time < time+1:
                    if packet.haslayer(sp.UDP):
                        udp.append(packet)
                    else:
                        non_udp.append(packet)
                    indexes[i] +=1
                else:
                    break


          
            udps+=len(udp)
        #### We make the assumption here that udp packets are urgent and need to be sent immediately so we do not introduce delays for udp
            return_packets = return_packets + udp 
            num_injected += len(udp)
        
        ####### queuing the non udp packets
            dbs[i] = dbs[i]+non_udp
        print("udp")
        print(udps)
            
            
            
#         #### we are assuming that the non_udp packets cannot wait for more than delay_time seconds, 
#         ####after delay_time second, any outstanding packets have to be sent  
#         ###### we will do that for each device
        for i in range(num_devices):
            pops = 0
            for packet in dbs[i]:
                if time - packet.time > delay_times[i]:
                    random_time = time + random.uniform(0,1)
                    packet.time = random_time
                    return_packets.append(packet)
                    num_injected +=1
                    pops+=1
            for j in range(pops):
                    dbs[i].pop(0)
        print("udp and urgent")
        print(num_injected)
                
# #         ###### we decide if we should do some more injection, 
#         ####### if the num_injected has not exceeded the upper threshold, we may inject more traffic
        if num_injected < packets_rates*(1+fluctuation_rate):
#         ############ it is a big decision how much to inject so that it both decrease the buffer as well as generate randomness#########

# #             ##### we decide randomly how much to inject, 
#             #######in doing this, we also take into account the length of the queue size
            injected_dbs = random.randint(math.floor(packets_rates*(1-fluctuation_rate)),math.floor(packets_rates))  - num_injected  
            
            if injected_dbs>0:
                print("random injection")
                print(injected_dbs)
#                 ##### we should implement the injected_dbs to ensure fairness
#                 #### fairness will be in the round_robin way
                
                queue_num = 0
#                 #### the queue_num determines which queue the packets should take from
                for i in range(injected_dbs):
                    injected_successfully = False
                    for j in range(num_devices):
                        queue = (queue_num + j)%num_devices
                        if len(dbs[queue])>0:
                            packet = dbs[queue][0]
                            random_time = time + random.uniform(0,1)
                            packet.time = random_time
                            return_packets.append(packet)
                            dbs[queue].pop(0)
                            num_injected +=1
                            queue_num = (queue_num +1)%num_devices
                            injected_successfully = True
                            break
                    if injected_successfully == False:
                        break
                
                       


        dbs_len = sum([len(dbs[i]) for i in range(num_devices)])
        if dbs_len>0:
               print("need more injection")
#                 #             #### if the buffer is too full, we will inject more
            if dbs_len> queue_size * packets_rates:

                injected_dbs = min(dbs_len-int(queue_size * packets_rates),math.floor(packets_rates*(1+fluctuation_rate))-num_injected) 

            else:
                injected_dbs = int(queue_size * dbs_len)
            print(injected_dbs)
            if injected_dbs>0:
                queue_num = 0
#                 #### the queue_num determines which queue the packets should take from
                for i in range(injected_dbs):
                    injected_successfully = False
                    for j in range(num_devices):
                        queue = (queue_num + j)%num_devices
                        if len(dbs[queue])>0:
                            packet = dbs[queue][0]
                            random_time = time + random.uniform(0,1)
                            packet.time = random_time
                            return_packets.append(packet)
                            dbs[queue].pop(0)
                            num_injected +=1
                            queue_num = (queue_num +1)%num_devices
                            injected_successfully = True
                            break
                    if injected_successfully == False:
                        break

        if num_injected <packets_rates*(1-fluctuation_rate):
            inject_cbs = random.randint(int(packets_rates*(1-fluctuation_rate)), int(packets_rates)) - num_injected

            for i in range(inject_cbs):
                packet = cbs[0].copy()
                random_time = time + random.uniform(0,1)
                packet.time = random_time
                return_packets.append(packet)
                num_injected+=1




                    
                
                    
                    
                    
        injected_packets_num.append(num_injected)
    
       
                
       
    return  injected_packets_num, return_packets 
                
  


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 117)

In [127]:
# def main():
    
    #### here we will use 3 different devices, alexa, nestcam and refrigerator
    

### Getting the average rate for each device
ave_packets_alexa = determining_normal_rate_packets("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/alexa/play_music/2/play_music_3_1614040169.pcap")
ave_packets_fridge = determining_normal_rate_packets("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/refrigerator/screen_interaction/2/screen_interaction_2_1625874845.pcap")
ave_packets_nestcam = determining_normal_rate_packets("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/nestcam/trigger_motion_detection/2/trigger_motion_detection_8_1614903296.pcap")

###### arranging the incoming packets by arrival time to simulate a real life scenario
packets_sorted_alexa = arrange_by_arrival_time("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/alexa/play_music/2/play_music_3_1614040169.pcap")
time_start_alexa = packets_sorted_alexa[0].time    
packets_sorted_fridge = arrange_by_arrival_time("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/refrigerator/screen_interaction/2/screen_interaction_2_1625874845.pcap")
time_start_fridge = packets_sorted_fridge[0].time
adjust_fridge = time_start_alexa -time_start_fridge

for packet in packets_sorted_fridge:
    packet.time += adjust_fridge


packets_sorted_nestcam = arrange_by_arrival_time("/Users/vantran/Desktop/GRAD_RESEARCH/Shinan/IOT-data/data-clean/nestcam/trigger_motion_detection/2/trigger_motion_detection_8_1614903296.pcap")
time_start_nestcam = packets_sorted_nestcam[0].time
adjust_nestcam = time_start_alexa - time_start_nestcam

for packet in packets_sorted_nestcam:
    packet.time += adjust_nestcam
    
    
    
    
########## make the device have the same start time and end_time #######
time_end_alexa = packets_sorted_alexa[-1].time
time_end_fridge = packets_sorted_fridge[-1].time
time_end_nestcam = packets_sorted_nestcam[-1].time
time_end_simulteneous = min(time_end_alexa, time_end_fridge,time_end_nestcam )

continue_remove = True 
while continue_remove:

    if packets_sorted_alexa[-1].time >time_end_simulteneous:
        packets_sorted_alexa.pop(-1)
    else:
        break
        
while continue_remove:

    if packets_sorted_fridge[-1].time >time_end_simulteneous:
        packets_sorted_fridge.pop(-1)
    else:
        break
        
while continue_remove:

    if packets_sorted_nestcam[-1].time >time_end_simulteneous:
        packets_sorted_nestcam.pop(-1)
    else:
        break
        
    
        
    









#     ##### if we want to increase the level of padding to ensure more protection, we can increase the ave_packets as well as decrease the fluctuation rate


    
   
    

    

    
  


In [128]:
fluctuation_rate = 0.3
queue_size = 0.4
delay_times = [7,5,6]
# #      ####### Van will vary the constants to see what will be the effect of varying these variables
buffer_levels = [ave_packets_alexa *1.6,ave_packets_fridge *1.6, ave_packets_nestcam*1.6]
packets_sorted = [packets_sorted_alexa,packets_sorted_fridge, packets_sorted_nestcam ]

injected_packets_num, return_packets  = simulate_by_packets(sum(buffer_levels), packets_sorted, fluctuation_rate,queue_size,delay_times)
os.system("echo Writing out buffered pcaps...")
sp.wrpcap('result.pcap', return_packets)
os.system("echo Reordering Pcap...")
os.system("reordercap -n {} {}".format("result.pcap","result_sorted_V.pcap"))



time
0
udp
18
udp
79
udp
97
udp and urgent
194
random injection
961
time
1
udp
110
udp
412
udp
914
udp and urgent
1436
time
2
udp
105
udp
538
udp
532
udp and urgent
1175
random injection
104
time
3
udp
58
udp
394
udp
447
udp and urgent
899
random injection
493
time
4
udp
190
udp
470
udp
396
udp and urgent
1056
random injection
151
time
5
udp
166
udp
333
udp
420
udp and urgent
919
random injection
312
time
6
udp
80
udp
514
udp
463
udp and urgent
1057
random injection
410
time
7
udp
77
udp
380
udp
415
udp and urgent
872
random injection
472
time
8
udp
83
udp
469
udp
410
udp and urgent
962
random injection
156
time
9
udp
64
udp
369
udp
381
udp and urgent
814
random injection
277
time
10
udp
52
udp
447
udp
485
udp and urgent
984
random injection
151
time
11
udp
58
udp
326
udp
413
udp and urgent
797
random injection
556
time
12
udp
132
udp
475
udp
452
udp and urgent
1059
random injection
66
time
13
udp
66
udp
377
udp
431
udp and urgent
874
random injection
347
time
14
udp
95
udp
442
udp
438

0

In [129]:
print(injected_packets_num)

[1102, 1507, 1319, 1118, 1263, 1197, 1329, 1470, 1342, 1188, 1208, 1133, 1164, 1125, 1152, 1227, 1194, 1083, 1111, 1050, 1470, 1485, 1079, 1039, 1277, 1175, 1180, 1067, 1141, 1236, 1419, 1097, 1268, 1279, 1130, 1248, 1423]


In [130]:
ave_packets = [ave_packets_alexa, ave_packets_fridge, ave_packets_nestcam]
evaluate_buffer(sum(ave_packets), injected_packets_num, 1.6, fluctuation_rate )

(0.3184364106523319, 1.0)